# 时间序列

时间序列数据是许多不同领域的结构化数据的重要形式，例如金融，经济学，生态学，神经科学和物理学。在许多时间点上观察或测量的任何东西都会形成一个时间序列。许多时间序列是固定频率的，也就是说，数据点根据某些规则以固定的间隔出现，例如每15秒，每5分钟或每月一次。时间序列也可以是不规则的，没有固定的时间单位或单位之间的偏移。如何标记和引用时间序列数据取决于应用程序，并且您可能具有以下之一：

- 时间戳记，特定时间点
- 固定时间段，例如2007年1月或2010年全年
- 时间间隔，以开始和结束时间戳记表示。句点可以被视为间隔的特殊情况
- 实验或经过的时间；每个时间戳是相对于特定开始时间的时间度量（例如，自放入烤箱以来每秒烘烤的曲奇的直径）

在本章中，我主要关注前三类中的时间序列，尽管许多技术都可以应用于实验时间序列，其中索引可以是整数或浮点数，指示从实验开始经过的时间。最简单和使用最广泛的时间序列是通过时间戳索引的时间序列。

> pandas还支持基于时间增量的索引，这可能是表示实验或经过时间的有用方式。 我们不会在本书中探讨timedelta索引，但是您可以在pandas文档中了解更多信息。

熊猫提供了许多内置的时间序列工具和数据算法。您可以有效地处理非常大的时间序列，并轻松地对不规则和固定频率的时间序列进行切片和切块，聚合以及重新采样。其中一些工具对于金融和经济应用程序特别有用，但是您当然也可以使用它们来分析服务器日志数据。

## 11.1 日期和时间数据类型和工具

Python标准库包括日期和时间数据的数据类型，以及与日历相关的功能。日期时间，时间和日历模块是开始的主要位置。datetime.datetime类型，或简称为datetime，被广泛使用：

In [1]:
from datetime import datetime

now = datetime.now()

now

datetime.datetime(2020, 7, 15, 22, 11, 47, 738938)

In [2]:
now.year, now.month, now.day

(2020, 7, 15)

datetime将日期和时间都存储到微秒。timedelta表示两个日期时间对象之间的时间差：

In [3]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)

delta

datetime.timedelta(days=926, seconds=56700)

In [4]:
delta.days

926

In [5]:
delta.seconds

56700

您可以向日期时间对象添加（或减去）一个时间增量或多个时间增量，以产生一个新的移位对象：

In [6]:
from datetime import timedelta

In [7]:
start = datetime(2011, 1, 7)

In [8]:
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [9]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

表11-1总结了datetime模块中的数据类型。尽管本章主要关注熊猫中的数据类型和更高级的时间序列操作，但您可能在Python的许多其他地方遇到了基于日期时间的类型。

### 在字符串和日期时间之间转换

您可以使用str或strftime方法格式化日期时间对象和pandas时间戳对象（稍后将介绍）为字符串，并传递格式说明：

In [10]:
stamp = datetime(2011, 1, 3)

In [11]:
str(stamp)

'2011-01-03 00:00:00'

In [12]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

有关格式代码的完整列表，请参见表11-2（从第2章中摘录）。

您可以使用以下相同的格式代码使用datetime.strptime将字符串转换为日期：

In [13]:
value = '2011-01-03'

In [14]:
datetime.strptime(value, '%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [15]:
datestrs = ['7/6/2011', '8/6/2011']

In [16]:
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

datetime.strptime是解析已知格式的日期的好方法。但是，每次都要编写格式规范可能会有些烦人，尤其是对于常见的日期格式。在这种情况下，您可以在第三方dateutil包中使用parser.parse方法（在安装熊猫时会自动安装该方法）：

In [17]:
from dateutil.parser import parse

In [18]:
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

dateutil能够解析大多数人类难以理解的日期表示形式：

In [19]:
parse('Jan 31, 1997 10:45 PM')

datetime.datetime(1997, 1, 31, 22, 45)

在国际语言环境中，出现在月份之前的日期很常见，因此您可以通过dayfirst = True来表明这一点：

In [20]:
parse('6/12/2011', dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

pandas通常面向使用日期数组，无论是用作轴索引还是用作DataFrame中的列。to_datetime方法解析许多不同类型的日期表示形式。可以快速解析ISO 8601之类的标准日期格式：

In [21]:
datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']

In [22]:
import pandas as pd

pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

它还处理应视为缺失的值（无，空字符串等）：

In [23]:
idx = pd.to_datetime(datestrs + [None])

In [24]:
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [25]:
idx[2]

NaT

In [26]:
pd.isnull(idx)

array([False, False,  True])

NaT（不是时间）是时间戳数据的熊猫的null值。

> 警告：dateutil.parser是有用但不完善的工具。值得注意的是，它会将某些字符串识别为您可能不希望使用的日期，例如，“ 42”将被解析为具有今天日历日期的2042年。

datetime对象还具有许多其他国家或地区的语言的特定于区域设置的格式选项。例如，与英语系统相比，德语或法语系统上的缩写月份名称将有所不同。有关列表，请参见表11-3。

## 11.2 时间序列基础

pandas中一种基本的时间序列对象是一个以时间戳为索引的序列，该序列通常在pandas外部以Python字符串或日期时间对象表示：

In [27]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]

In [28]:
import numpy as np

ts = pd.Series(np.random.randn(6), index=dates)

In [29]:
ts

2011-01-02   -0.660864
2011-01-05   -0.892516
2011-01-07   -0.261048
2011-01-08    0.394434
2011-01-10    0.140635
2011-01-12    0.720481
dtype: float64

在后台，这些datetime对象已放入DatetimeIndex中：

In [30]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

与其他系列一样，索引不同的时间序列之间的算术运算会自动与日期对齐：

In [31]:
ts + ts[::2]

2011-01-02   -1.321728
2011-01-05         NaN
2011-01-07   -0.522096
2011-01-08         NaN
2011-01-10    0.281271
2011-01-12         NaN
dtype: float64

回想一下ts[:: 2]选择ts中的第二个元素。

大熊猫使用NumPy的datetime64数据类型以十亿分之一秒的分辨率存储时间戳：

In [32]:
ts.index.dtype

dtype('<M8[ns]')

DatetimeIndex中的标量值是pandas Timestamp对象：

In [33]:
stamp = ts.index[0]

In [34]:
stamp

Timestamp('2011-01-02 00:00:00')

可以在使用datetime对象的任何地方替换时间戳。此外，它可以存储频率信息（如果有）并了解如何进行时区转换和其他类型的操作。稍后将进一步讨论这两个方面。

### 索引，选择，子集

当您基于标签索引和选择数据时，时间序列的行为与任何其他pandas.Series一样。

In [35]:
stamp = ts.index[2]

In [36]:
ts[stamp]

-0.2610479931069228

为方便起见，您还可以传递可解释为日期的字符串：

In [37]:
ts['1/10/2011']

0.14063546468165292

In [38]:
ts['20110110']

0.14063546468165292

对于更长的时间序列，可以传递一年或仅一年零一个月来轻松选择数据切片：

In [39]:
longer_ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))

In [40]:
longer_ts

2000-01-01   -0.006326
2000-01-02    0.552131
2000-01-03    1.406631
2000-01-04    1.028800
2000-01-05   -1.031235
                ...   
2002-09-22    2.625855
2002-09-23    0.285741
2002-09-24   -1.965145
2002-09-25   -1.784176
2002-09-26   -1.250172
Freq: D, Length: 1000, dtype: float64

In [41]:
longer_ts['2001']

2001-01-01    0.476914
2001-01-02   -0.753122
2001-01-03    0.924799
2001-01-04   -2.727020
2001-01-05    0.802804
                ...   
2001-12-27    0.037122
2001-12-28    0.335198
2001-12-29    1.301071
2001-12-30    0.085082
2001-12-31   -1.231003
Freq: D, Length: 365, dtype: float64

在这里，字符串“ 2001”被解释为年份，并选择该时间段。如果您指定月份，这也可以使用：

In [42]:
longer_ts['2001-05']

2001-05-01   -0.997129
2001-05-02   -1.842631
2001-05-03    0.423358
2001-05-04    0.994099
2001-05-05    0.601493
2001-05-06    0.789656
2001-05-07   -0.574008
2001-05-08   -0.069458
2001-05-09   -0.042415
2001-05-10   -0.908153
2001-05-11   -0.338875
2001-05-12    2.214150
2001-05-13    0.272227
2001-05-14   -0.197819
2001-05-15    0.619748
2001-05-16    0.066555
2001-05-17   -0.440852
2001-05-18   -0.555304
2001-05-19    1.694570
2001-05-20   -0.912564
2001-05-21   -0.707604
2001-05-22    1.061711
2001-05-23    1.157975
2001-05-24   -0.898889
2001-05-25    1.220020
2001-05-26   -0.092083
2001-05-27   -0.125201
2001-05-28    0.981446
2001-05-29    0.993833
2001-05-30   -0.722777
2001-05-31   -0.301047
Freq: D, dtype: float64

用datetime对象切片也可以：

In [43]:
ts[datetime(2011, 1, 7)]

-0.2610479931069228

由于大多数时间序列数据都是按时间顺序排列的，因此可以使用时间序列中未包含的时间戳进行切片以执行范围查询：

In [44]:
ts

2011-01-02   -0.660864
2011-01-05   -0.892516
2011-01-07   -0.261048
2011-01-08    0.394434
2011-01-10    0.140635
2011-01-12    0.720481
dtype: float64

In [45]:
ts['1/6/2011':'1/11/2011']

2011-01-07   -0.261048
2011-01-08    0.394434
2011-01-10    0.140635
dtype: float64

和以前一样，您可以传递字符串日期，日期时间或时间戳。请记住，以这种方式进行切片会像切片NumPy数组一样在源时间序列上产生视图。这意味着不会复制任何数据，并且片上的修改将反映在原始数据中。

有一个等效的实例方法truncate，它在两个日期之间切片一个Series：

In [46]:
ts.truncate(after='1/9/2011')

2011-01-02   -0.660864
2011-01-05   -0.892516
2011-01-07   -0.261048
2011-01-08    0.394434
dtype: float64

所有这些对于DataFrame同样适用，在其行上建立索引：

In [47]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')

In [48]:
long_df = pd.DataFrame(np.random.randn(100, 4),
                       index=dates,
                       columns=['Colorado', 'Texas', 'New York', 'Ohio'])

In [49]:
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,-0.315126,-1.785623,0.256569,0.061982
2001-05-09,1.038342,-0.887821,1.051129,-0.350965
2001-05-16,0.877792,-0.180605,1.490712,-1.342175
2001-05-23,-0.230326,0.025445,1.239237,0.434418
2001-05-30,-0.249859,-1.176816,-0.777604,-1.397981


### 具有重复索引的时间序列

在某些应用程序中，可能会有多个数据观测值落在特定的时间戳上。这是一个例子：

In [50]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])

In [51]:
dup_ts = pd.Series(np.arange(5), index=dates)

In [52]:
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

我们可以通过检查其is_unique属性来判断索引不是唯一的：

In [53]:
dup_ts.index.is_unique

False

现在，根据是否复制时间戳，索引到该时间序列将产生标量值或分片：

In [54]:
dup_ts['1/3/2000']

4

In [55]:
dup_ts['1/2/2000']

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

假设您要聚合具有非唯一时间戳的数据。一种方法是使用groupby并通过level = 0：

In [56]:
grouped = dup_ts.groupby(level=0)

In [57]:
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

In [58]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 11.3 日期范围，频率和移位

pandas的一般时间序列被认为是不规则的；也就是说，它们没有固定的频率。对于许多应用来说，这已经足够了。但是，通常最好相对于固定频率（例如每天，每月或每15分钟）进行工作，即使这意味着在时间序列中引入缺失值。幸运的是，pandas拥有一套完整的标准时间序列频率以及用于重新采样，推断频率并生成固定频率日期范围的工具。例如，您可以通过调用resample将采样时间序列转换为固定的每日频率：

In [59]:
ts

2011-01-02   -0.660864
2011-01-05   -0.892516
2011-01-07   -0.261048
2011-01-08    0.394434
2011-01-10    0.140635
2011-01-12    0.720481
dtype: float64

In [60]:
resampler = ts.resample('D')

字符串“ D”被解释为每日频率。

频率之间的转换或重采样是一个足够大的主题，稍后可以在其自己的部分中找到（第348页的第11.6节“重采样和频率转换”）。在这里，我将向您展示如何使用基本频率及其倍数。

### 生成日期范围

虽然我以前没有解释就使用了它，但pandas.date_range负责根据特定频率生成具有指定长度的DatetimeIndex：

In [62]:
index = pd.date_range('2012-04-01', '2012-06-01')

In [63]:
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

默认情况下，date_range生成每日时间戳记。如果仅传递开始日期或结束日期，则必须传递多个期间才能生成：

In [64]:
pd.date_range(start='2012-04-01', periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [65]:
pd.date_range(end='2012-06-01', periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

开始日期和结束日期为生成的日期索引定义了严格的边界。例如，如果您想要一个包含每个月最后一个工作日的日期索引，则可以传递“ BM”频率（一个月的营业时间；在表11-4中查看频率的更完整列表），并且仅在或 日期间隔内将包括：

In [66]:
pd.date_range('2000-01-01', '2000-12-01', freq='BM')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BM')

默认情况下，date_range保留开始或结束时间戳记的时间（如果有）：

In [67]:
pd.date_range('2012-05-02 12:56:31', periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

有时，您将具有带有时间信息的开始或结束日期，但希望生成一组标准化为午夜的时间戳作为惯例。为此，有一个规范化选项：

In [68]:
pd.date_range('2012-05-02 12:56:31', periods=5, normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### 频率和日期偏移

熊猫的频率由基本频率和乘数组成。基本频率通常由字符串别名表示，例如“ M”代表每月，“ H”代表每小时。对于每个基本频率，有一个对象通常被定义为日期偏移量。例如，小时频率可以用Hour类表示：

In [69]:
from pandas.tseries.offsets import Hour, Minute

In [70]:
hour = Hour()

In [71]:
hour

<Hour>

您可以通过传递整数来定义偏移量的倍数：

In [72]:
four_hours = Hour(4)

In [73]:
four_hours

<4 * Hours>

在大多数应用程序中，您将不需要显式创建这些对象之一，而只需使用字符串别名（例如“ H”或“ 4H”）即可。 在基频之前放置一个整数会创建一个倍数：

In [77]:
pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

许多偏移量可以通过加在一起组合在一起：

In [78]:
Hour(2) + Minute(30)

<150 * Minutes>

同样，您可以传递频率字符串，例如“ 1h30min”，这些字符串将有效地解析为相同的表达式：

In [79]:
pd.date_range('2000-01-01', periods=10, freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

某些频率描述的时间点间隔不均匀。例如，“ M”（日历的月末）和“ BM”（月的最后一个工作日/工作日）取决于一个月中的天数，在后者的情况下，则取决于该月是否在周末结束。我们称这些为锚定偏移量。

返回表11-4，以获取熊猫中可用的频率代码和日期偏移量类别的列表。

用户可以定义自己的自定义频率类别，以提供大熊猫无法使用的日期逻辑，尽管其详细信息不在本书的讨论范围之内。

#### 一个月中的第几周

从WOM开始，一种有用的频率类别是“每月的一周”。 这样一来，您便可以获取每个月的第三个星期五等日期：

In [80]:
rng = pd.date_range('2012-01-01', '2012-09-01', freq='WOM-3FRI')

In [82]:
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

### 移位（超前和滞后）数据

“移位”是指在时间上前后移动数据。Series和DataFrame都有一个shift方法，用于向前或向后进行幼稚的移位，而索引保持不变：

In [83]:
ts = pd.Series(np.random.randn(4),
               index=pd.date_range('1/1/2000', periods=4, freq='M'))

In [84]:
ts

2000-01-31   -0.430259
2000-02-29   -0.331127
2000-03-31   -0.407736
2000-04-30    0.506452
Freq: M, dtype: float64

In [85]:
ts.shift(2)

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.430259
2000-04-30   -0.331127
Freq: M, dtype: float64

In [86]:
ts.shift(-2)

2000-01-31   -0.407736
2000-02-29    0.506452
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

当我们这样移动时，缺少的数据会在时间序列的开始或结尾处引入。

移位的常见用法是将一个时间序列或多个时间序列的变化百分比作为DataFrame列计算。 表示为：

In [87]:
ts / ts.shift(1) - 1

2000-01-31         NaN
2000-02-29   -0.230400
2000-03-31    0.231358
2000-04-30   -2.242108
Freq: M, dtype: float64

由于天真的变化会使索引保持不变，因此会丢弃某些数据。 因此，如果频率是已知的，则可以传递它以进行移位而不是简单地将数据提前到时间戳：

In [88]:
ts.shift(2, freq='M')

2000-03-31   -0.430259
2000-04-30   -0.331127
2000-05-31   -0.407736
2000-06-30    0.506452
Freq: M, dtype: float64

也可以传递其他频率，为您提供领先和滞后数据的灵活性：

In [89]:
ts.shift(3, freq='D')

2000-02-03   -0.430259
2000-03-03   -0.331127
2000-04-03   -0.407736
2000-05-03    0.506452
dtype: float64

In [90]:
ts.shift(1, freq='90T')

2000-01-31 01:30:00   -0.430259
2000-02-29 01:30:00   -0.331127
2000-03-31 01:30:00   -0.407736
2000-04-30 01:30:00    0.506452
Freq: M, dtype: float64

#### 偏移日期

大熊猫的日期偏移量也可以与datetime或Timestamp对象一起使用：

In [91]:
from pandas.tseries.offsets import Day, MonthEnd

In [92]:
now = datetime(2011, 11, 17)

In [93]:
now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

如果添加锚定的偏移量（如MonthEnd），则第一个增量将根据频率规则将日期“前滚”至下一个日期：

In [94]:
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [95]:
now + MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

锚定的偏移量可以分别通过分别使用前滚和后滚方法显式地“向前”滚动日期或向后滚动：

In [96]:
offset = MonthEnd()

In [97]:
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [98]:
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

创造性地使用日期偏移量是将这些方法与groupby一起使用：

In [99]:
ts = pd.Series(np.random.randn(20), index=pd.date_range('1/15/2000', periods=20, freq='4d'))

In [100]:
ts

2000-01-15   -1.405584
2000-01-19    0.687700
2000-01-23   -0.518135
2000-01-27   -0.466886
2000-01-31   -0.314746
2000-02-04   -0.859740
2000-02-08    0.255674
2000-02-12   -1.050324
2000-02-16   -0.033687
2000-02-20    1.978054
2000-02-24   -0.632769
2000-02-28   -1.482840
2000-03-03   -0.671477
2000-03-07    1.362919
2000-03-11   -0.191519
2000-03-15   -0.883145
2000-03-19   -0.804877
2000-03-23   -1.190349
2000-03-27   -0.236337
2000-03-31    0.087582
Freq: 4D, dtype: float64

In [101]:
ts.groupby(offset.rollforward).mean()

2000-01-31   -0.403530
2000-02-29   -0.260804
2000-03-31   -0.315900
dtype: float64

当然，一种更简便，更快速的方法是使用重采样（我们将在第348页的第11.6节“重采样和频率转换”中对此进行更深入的讨论）：

In [102]:
ts.resample('M').mean()

2000-01-31   -0.403530
2000-02-29   -0.260804
2000-03-31   -0.315900
Freq: M, dtype: float64

## 11.4 时区处理

通常，使用时区被认为是时间序列操作中最不愉快的部分之一。结果，许多时间序列用户选择使用协调世界时或UTC的时间序列，这是格林威治标准时间的后继者，并且是当前的国际标准。时区表示为与UTC的偏移量；例如，在夏令时期间，纽约比世界协调时间晚四个小时，而在一年中的其他时间则落后五个小时。

在Python中，时区信息来自第三方pytz库（可通过pip或conda安装），该库公开了世界时区信息的汇编Olson数据库。这对于历史数据尤为重要，因为夏令时（DST）转换日期（甚至UTC偏移）已根据地方政府的异想而过多次更改。在美国，自1900年以来，DST转换时间已多次更改！

有关pytz库的详细信息，您需要查看该库的文档。就本书而言，pandas包装了pytz的功能，因此您可以在时区名称之外忽略其API。时区名称可以在文档中以交互方式找到：

In [104]:
import pytz

pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

要从pytz获取时区对象，请使用pytz.timezone：

In [109]:
tz = pytz.timezone('America/New_York')

tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

熊猫中的方法将接受时区名称或这些对象。

### 时区本地化和转换

默认情况下，熊猫中的时间序列是时区幼稚的。 例如，考虑以下时间序列：

In [114]:
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')

In [116]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [117]:
ts

2012-03-09 09:30:00   -0.120738
2012-03-10 09:30:00   -0.653282
2012-03-11 09:30:00    0.465504
2012-03-12 09:30:00    0.788703
2012-03-13 09:30:00   -2.239855
2012-03-14 09:30:00   -1.638225
Freq: D, dtype: float64

索引的tz字段为None：

In [119]:
print(ts.index.tz)

None


可以设置时区来生成日期范围：

In [121]:
pd.date_range('3/9/2012 9:30', periods=10, freq='D', tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

从朴素到本地化的转换由tz_localize方法处理：

In [123]:
ts

2012-03-09 09:30:00   -0.120738
2012-03-10 09:30:00   -0.653282
2012-03-11 09:30:00    0.465504
2012-03-12 09:30:00    0.788703
2012-03-13 09:30:00   -2.239855
2012-03-14 09:30:00   -1.638225
Freq: D, dtype: float64

In [124]:
ts_utc = ts.tz_localize('America/New_York')

In [125]:
ts_utc

2012-03-09 09:30:00-05:00   -0.120738
2012-03-10 09:30:00-05:00   -0.653282
2012-03-11 09:30:00-04:00    0.465504
2012-03-12 09:30:00-04:00    0.788703
2012-03-13 09:30:00-04:00   -2.239855
2012-03-14 09:30:00-04:00   -1.638225
Freq: D, dtype: float64

对于前面的时间序列，它跨越了America / New_York时区的DST转换，我们可以本地化为EST并转换为UTC或柏林时间：

In [126]:
ts_eastern = ts.tz_localize('America/New_York')
ts_eastern.tz_convert('UTC')

2012-03-09 14:30:00+00:00   -0.120738
2012-03-10 14:30:00+00:00   -0.653282
2012-03-11 13:30:00+00:00    0.465504
2012-03-12 13:30:00+00:00    0.788703
2012-03-13 13:30:00+00:00   -2.239855
2012-03-14 13:30:00+00:00   -1.638225
Freq: D, dtype: float64

In [127]:
ts_eastern.tz_convert('Europe/Berlin')

2012-03-09 15:30:00+01:00   -0.120738
2012-03-10 15:30:00+01:00   -0.653282
2012-03-11 14:30:00+01:00    0.465504
2012-03-12 14:30:00+01:00    0.788703
2012-03-13 14:30:00+01:00   -2.239855
2012-03-14 14:30:00+01:00   -1.638225
Freq: D, dtype: float64

tz_localize和tz_convert也是DatetimeIndex的实例方法：

In [128]:
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq='D')

> 警告：本地化朴素的时间戳还将检查夏时制时间转换周围是否存在模棱两可或不存在的时间。

### 使用时区感知时间戳对象的操作

与时间序列和日期范围类似，可以将单个Timestamp对象类似地从原始位置本地化到时区，并将其从一个时区转换为另一个时区：

In [129]:
stamp = pd.Timestamp('2011-03-12 04:00')

In [130]:
stamp_utc = stamp.tz_localize('utc')

In [134]:
stamp_utc.tz_convert('America/New_York')

Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

您还可以在创建时间戳时传递时区：

In [135]:
stamp_moscow = pd.Timestamp('2011-03-12 04:00', tz='Europe/Moscow')

In [136]:
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

自Unix时代（1970年1月1日）以来，可识别时区的Timestamp对象在内部存储UTC时间戳值（以纳秒为单位）。此UTC值在时区转换之间是不变的：

In [137]:
stamp_utc.value

1299902400000000000

In [139]:
stamp_utc.tz_convert('America/New_York').value

1299902400000000000

当使用熊猫的DateOffset对象执行时间算术运算时，熊猫会尽可能考虑夏令时的转换。在这里，我们构造在DST转换（向前和向后）之前发生的时间戳。首先，在转换为DST之前30分钟：

In [142]:
from pandas.tseries.offsets import Hour

In [143]:
stamp = pd.Timestamp('2012-03-12 01:30', tz='US/Eastern')

In [144]:
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [145]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

然后，在退出DST之前90分钟：

In [146]:
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')

In [147]:
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [148]:
stamp + 2 * Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

### 不同时区之间的操作

如果将两个具有不同时区的时间序列组合在一起，则结果将是UTC。由于时间戳记以UTC形式存储在幕后，因此这是一种简单的操作，不需要进行任何转换：

In [149]:
rng = pd.date_range('3/7/2012 9:30', periods=10, freq='B')

In [151]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [152]:
ts

2012-03-07 09:30:00   -0.765489
2012-03-08 09:30:00   -1.769964
2012-03-09 09:30:00    1.078446
2012-03-12 09:30:00    0.200598
2012-03-13 09:30:00    0.158352
2012-03-14 09:30:00   -0.198946
2012-03-15 09:30:00   -0.817841
2012-03-16 09:30:00   -1.471774
2012-03-19 09:30:00    0.816290
2012-03-20 09:30:00    1.935733
Freq: B, dtype: float64

In [153]:
ts1 = ts[:7].tz_localize('Europe/London')

In [156]:
ts2 = ts1[2:].tz_convert('Europe/Moscow')

In [159]:
result = ts1 + ts2

In [160]:
result

2012-03-07 09:30:00+00:00         NaN
2012-03-08 09:30:00+00:00         NaN
2012-03-09 09:30:00+00:00    2.156893
2012-03-12 09:30:00+00:00    0.401197
2012-03-13 09:30:00+00:00    0.316704
2012-03-14 09:30:00+00:00   -0.397893
2012-03-15 09:30:00+00:00   -1.635682
Freq: B, dtype: float64

In [161]:
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')